- [Linear Regression](#linear-regression)
- [Logistic Regression + MCMC + Moment Matching](#Logistic-Regression-+-MCMC-+-Moment-Matching)
- [Logistic Regression + MCMC](#Logistic-Regression-+-MCMC)
- [Logistic Regression + PP Posterior Update](#Logistic-Regression-+-PP-Posterior-Update)
- [Online IRT MCMC](#Online-IRT-MCMC)
- [Batch IRT Search](#Online-IRT-Search)

In [1]:
import probpy as pp
import numpy as np
import numba
import random

linear regression
---

In [2]:
%%time
def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]

w = [5, -2, 1, 5] # True underlying model
x = np.random.rand(100, 3) * 10
y = predict(w, x) + pp.normal.sample(mu=0, sigma=1, size=100).reshape(-1)



prior = pp.multivariate_normal.med(mu=np.ones(4) * 0, sigma=np.eye(4) * 5)
likelihood = pp.unilinear.med(sigma=1) # There exist an implementation for linear because it has a conjugate prior

for i in range(100):
    data = (y[i], x[i])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, priors=prior)
    
    if i % 10 == 0:
        w_approx = pp.mode(prior)
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - predict(w_approx, x)).mean(), 
              "True MSE", np.square(y - predict(w, x)).mean())
        print()

Parameter Estimate [1.99559744 1.83237212 1.10250381 0.23260712]
Prior MSE 203.39502472928478 True MSE 0.9551699070299853

Parameter Estimate [ 4.91796147 -1.82791172  1.20754261  3.16726393]
Prior MSE 1.89642851192347 True MSE 0.9551699070299853

Parameter Estimate [ 4.86357297 -1.91882655  1.09823064  4.85695074]
Prior MSE 1.3143294969173605 True MSE 0.9551699070299853

Parameter Estimate [ 4.93935868 -1.97729392  1.09073202  4.69988922]
Prior MSE 1.1004105792790415 True MSE 0.9551699070299853

Parameter Estimate [ 4.98147525 -2.01858915  1.00324045  4.93313186]
Prior MSE 0.9617383452332996 True MSE 0.9551699070299853

Parameter Estimate [ 4.95876918 -2.01392115  0.97475037  5.14642372]
Prior MSE 0.9550987321236315 True MSE 0.9551699070299853

Parameter Estimate [ 4.95068124 -1.98680109  0.97293979  5.04113828]
Prior MSE 0.9668346141338117 True MSE 0.9551699070299853

Parameter Estimate [ 4.97442319 -1.99645419  0.96960143  5.0044558 ]
Prior MSE 0.9504814651025012 True MSE 0.95516990

Logistic Regression + MCMC + Moment Matching
---

In [3]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior

def likelihood(y, x, w):
    return pp.normal.p((y - sigmoid(x @ w[:, :-1, None] + w[:, None, None, -1]).squeeze(axis=2)),
                    mu=0.0, sigma=1.0)
    
    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 5)

for i in range(50):
    j = random.randint(0, 80)
    data = (y[j: j + 20], x[j: j + 20])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   match_moments_for=pp.multivariate_normal,
                                   batch=30,
                                   samples=3000,
                                   mixing=200, 
                                   energies=1.0,
                                   mode="mcmc")

    if i % 10 == 0:
        w_approx = pp.mode(prior)
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
              "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
        print()

Parameter Estimate [-0.58368527 -0.00474065 -0.03402074 -0.98776543]
Prior MSE 0.15105877607130266 True MSE 0.027767242071072798

Parameter Estimate [-2.56034941  1.58538386  3.78681307 -1.79299206]
Prior MSE 0.02771556701244159 True MSE 0.027767242071072798

Parameter Estimate [-2.53660432  1.81536756  4.64343921 -2.12575396]
Prior MSE 0.026096644887211792 True MSE 0.027767242071072798

Parameter Estimate [-2.61084506  1.9958519   5.35246106 -2.55431431]
Prior MSE 0.027113792526636468 True MSE 0.027767242071072798

Parameter Estimate [-3.13104394  2.5968572   5.78705189 -2.98785927]
Prior MSE 0.02747016264800492 True MSE 0.027767242071072798

CPU times: user 15.4 s, sys: 23.2 s, total: 38.6 s
Wall time: 10.1 s


Logistic Regression + MCMC
---

In [15]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior

def likelihood(y, x, w):
    return pp.normal.p((y - sigmoid(x @ w[:, :-1, None] + w[:, None, None, -1]).squeeze(axis=2)),
                    mu=0.0, sigma=1.0)
    
    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 5)

for i in range(5):
    data = (y, x)
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   batch=5,
                                   samples=1000,
                                   mixing=100, 
                                   energies=0.3,
                                   mode="mcmc")

    modes = pp.mode(prior) # modes are sorted in order first is largest

    print("Number of modes", len(modes))
    w_approx = modes[0]

    print("Parameter Estimate", w_approx)

    print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
          "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
    print()

Number of modes 2
Parameter Estimate [-2.00014761  0.89643344  1.67774935 -0.86108738]
Prior MSE 0.040330932591864155 True MSE 0.021678924604256383

Number of modes 2
Parameter Estimate [-3.26034375  1.88801416  1.72241994 -0.48213026]
Prior MSE 0.03600645023741543 True MSE 0.021678924604256383

Number of modes 1
Parameter Estimate [-2.88598249  1.49255633  2.79466557 -0.8886015 ]
Prior MSE 0.02657179385683324 True MSE 0.021678924604256383

Number of modes 2
Parameter Estimate [-2.99678233  1.94050077  3.23657538 -1.03598188]
Prior MSE 0.02946572878396506 True MSE 0.021678924604256383

Number of modes 2
Parameter Estimate [-2.48285591  1.81581448  2.73841424 -1.63612152]
Prior MSE 0.02738771825835502 True MSE 0.021678924604256383

CPU times: user 1.27 s, sys: 818 ms, total: 2.09 s
Wall time: 1.19 s


Logistic Regression + Search
---


In [14]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior
def likelihood(y, x, w):
    return pp.normal.p((y - sigmoid(x @ w[:, :-1, None] + w[:, None, None, -1]).squeeze(axis=2)),
                    mu=0.0, sigma=1.0)
    
    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 10)

for i in range(5):
    data = (y, x)
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   batch=50,
                                   samples=1000,
                                   energies=0.25,
                                   mode="search",
                                   volume=1000)

    modes = pp.mode(prior) # modes are sorted in order first is largest

    
    print("Number of modes", len(modes))
    for mode in modes:
        print(mode)

    w_approx = modes[0]

    print("Parameter Estimate", w_approx)

    print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
          "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
    print()

Number of modes 5
[-2.09769567  1.62888091  2.83550617 -1.66533419]
[-1.70338861  1.4593331   3.57524701 -1.17629273]
[-2.21563849  1.94664001  3.61763174 -1.75253601]
[-1.26829558  1.14523862  3.69190497 -1.82670079]
[-1.85866349  1.49373765  3.82101128 -1.9803491 ]
Parameter Estimate [-2.09769567  1.62888091  2.83550617 -1.66533419]
Prior MSE 0.026647292517380086 True MSE 0.02380137662073885

Number of modes 2
[-1.8877228   1.50536392  2.75388995 -1.56296231]
[-2.05143615  1.62483292  3.29180334 -1.83670573]
Parameter Estimate [-1.8877228   1.50536392  2.75388995 -1.56296231]
Prior MSE 0.026301980245897038 True MSE 0.02380137662073885

Number of modes 2
[-1.94756153  1.64929901  2.81911065 -1.62939674]
[-2.12624048  1.32647163  2.79985965 -1.6073133 ]
Parameter Estimate [-1.94756153  1.64929901  2.81911065 -1.62939674]
Prior MSE 0.02538478344399076 True MSE 0.02380137662073885

Number of modes 1
[-2.08487297  1.65585549  2.86848708 -1.659646  ]
Parameter Estimate [-2.08487297  1.6558

Online-IRT MCMC
---

In [9]:
%%time

def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def logit(x):
    return np.log(x / (1 - x))

student_skill = logit(0.7)

items = logit(np.array([0.4, 0.6, 0.8, 0.7]))  # difficulties

def likelihood(obs, item, skill):
    result = []
    for _skill in skill:
        result.append(pp.normal.p(obs - sigmoid(_skill - item), mu=0.0, sigma=0.6))

    return np.array(result)

samples = 30
obs, its = [], []
for i in range(samples):  # 100 samples
    item = items[np.random.randint(0, items.size)]
    outcome = (np.random.rand() < sigmoid(student_skill - item)).astype(np.float)

    obs.append(outcome)
    its.append(item)

prior_skill = pp.normal.med(mu=0.0, sigma=10)

for i in range(samples):
    prior_skill = pp.parameter_posterior((obs[i], its[i]), likelihood=likelihood, priors=prior_skill,
                                         mode="mcmc", match_moments_for=pp.normal,
                                         samples=20000, mixing=3000, batch=5)
    modes = sigmoid(np.array(pp.mode(prior_skill)))
    
    
    print("observation", obs[i], "item", sigmoid(its[i]), "mode", sigmoid(pp.mode(prior_skill)))



observation 1.0 item 0.4 mode 0.6659695025664282
observation 1.0 item 0.8 mode 0.8020286414840695
observation 1.0 item 0.4 mode 0.8508628052739741
observation 1.0 item 0.4 mode 0.9129996094595377
observation 0.0 item 0.8 mode 0.8412823033867718
observation 0.0 item 0.6 mode 0.7354109825371952
observation 0.0 item 0.7 mode 0.6618174814469742
observation 0.0 item 0.8 mode 0.5871783987109122
observation 0.0 item 0.8 mode 0.5035837085155406
observation 0.0 item 0.6 mode 0.43600480550947657
observation 0.0 item 0.4 mode 0.3737703684795617
observation 1.0 item 0.4 mode 0.4503008206841502
observation 1.0 item 0.7 mode 0.5263954859032576
observation 0.0 item 0.7 mode 0.4853816985735765
observation 1.0 item 0.4 mode 0.5472069801893906
observation 1.0 item 0.8 mode 0.5895748042908497
observation 1.0 item 0.7 mode 0.6369822092748112
observation 1.0 item 0.7 mode 0.672791678985395
observation 1.0 item 0.6 mode 0.7008580224126745
observation 1.0 item 0.7 mode 0.7342480704115563
observation 0.0 item

Online IRT Search
---

In [11]:
%%time

def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def logit(x):
    return np.log(x / (1 - x))

student_skill = logit(0.7)

items = logit(np.array([0.4, 0.6, 0.8, 0.7]))  # difficulties

def likelihood(obs, item, skill):
    result = []
    for _skill in skill:
        result.append(pp.normal.p(obs - sigmoid(_skill - item), mu=0.0, sigma=0.6))

    return np.array(result)

samples = 30
obs, its = [], []
for i in range(samples):  # 100 samples
    item = items[np.random.randint(0, items.size)]
    outcome = (np.random.rand() < sigmoid(student_skill - item)).astype(np.float)

    obs.append(outcome)
    its.append(item)

prior_skill = pp.normal.med(mu=0.0, sigma=10)

for i in range(samples):
    prior_skill = pp.parameter_posterior((obs[i], its[i]), 
                                         likelihood=likelihood, priors=prior_skill,
                                         mode="search",
                                         samples=500, batch=5,
                                         volume=100, energy=0.1)
    modes = sigmoid(np.array(pp.mode(prior_skill)))
    
    print("observation", obs[i], "item", sigmoid(its[i]), "mode", sigmoid(pp.mode(prior_skill))[0])




observation 1.0 item 0.4 mode [0.66908634]
observation 1.0 item 0.4 mode [0.68402731]
observation 0.0 item 0.8 mode [0.67153789]
observation 1.0 item 0.6 mode [0.68855157]
observation 1.0 item 0.6 mode [0.70638245]
observation 1.0 item 0.4 mode [0.70899041]
observation 1.0 item 0.4 mode [0.71843775]
observation 0.0 item 0.6 mode [0.69275993]
observation 1.0 item 0.6 mode [0.70491049]
observation 0.0 item 0.7 mode [0.68388039]
observation 1.0 item 0.6 mode [0.69991047]
observation 1.0 item 0.6 mode [0.71534563]
observation 1.0 item 0.8 mode [0.73985352]
observation 0.0 item 0.6 mode [0.71149975]
observation 1.0 item 0.7 mode [0.73387809]
observation 0.0 item 0.7 mode [0.71202318]
observation 0.0 item 0.4 mode [0.68790718]
observation 0.0 item 0.8 mode [0.67527891]
observation 0.0 item 0.7 mode [0.65349838]
observation 1.0 item 0.6 mode [0.67431735]
observation 1.0 item 0.4 mode [0.68075407]
observation 0.0 item 0.7 mode [0.66447142]
observation 0.0 item 0.7 mode [0.64694849]
observation